In [4]:
import sys
!{sys.executable} -m pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 2.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.5 MB/s eta 0:00:0000:0100:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 2.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 1.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 3.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [1]:
import os
import cv2
import re
import random
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score
from sklearn.utils import shuffle

2024-08-25 19:07:11.194219: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-25 19:07:11.280829: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-25 19:07:11.365095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-25 19:07:11.432046: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-25 19:07:11.452114: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-25 19:07:11.581833: I tensorflow/core/platform/cpu_feature_gu

In [2]:
dataset_path = "histology_slides"  
regex_label = r"[A-Z]+_([A-Z])_[A-Z]+-\d{2}-[A-Z\d]+-(\d+)-\d+\.png"

In [3]:
# def load_data(dataset_path, img_size=(128,128)):

#     X = []
#     y = []

#     num_files = 0

#     for root, dirs, files in os.walk(dataset_path):
#         for file in files:
#             if file.endswith('.png'):

#                 img_path = os.path.join(root, file)
#                 img = cv2.imread(img_path)
#                 img = cv2.resize(img, img_size)
#                 X.append(img)

#                 # assumindo que label seja True ou False para maligno
#                 match_obj = re.search(regex_label, file)
#                 label = match_obj.group(1) # B(enigno) ou M(aligno)
#                 y.append(True if label == "M" else False)

#                 num_files = num_files + 1
#                 if num_files%500 == 0 and num_files != 0:
#                     print(str(num_files) + ": " + img_path)
#                     X = np.array(X)
#                     y = np.array(y)
#                     return X, y

#     print(num_files)

#     X = np.array(X)
#     y = np.array(y)
#     return X, y

In [3]:
def load_data(dataset_path, img_size=(128,128)):
    X = []
    y = []
    num_files = 0

    for root, dirs, files in os.walk(dataset_path):
        if os.path.basename(root) == "400X":  # Process only if it's a 40X folder
            for file in files:
                if file.endswith('.png'):
                    img_path = os.path.join(root, file)
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, img_size)
                    X.append(img)

                    # Assuming label is B(enigno) or M(aligno)
                    match_obj = re.search(regex_label, file)
                    label = match_obj.group(1)  # B or M
                    y.append(True if label == "M" else False)

                    num_files += 1

    print(f"Total images processed: {num_files}")

    X = np.array(X)
    y = np.array(y)
    return X, y

In [4]:
img_size = (700, 460)
X, y = load_data(dataset_path, img_size)

Total images processed: 1820


In [5]:
from collections import Counter

count = Counter(y)
print(count)

Counter({True: 1232, False: 588})


In [5]:
X = X / 255

In [5]:
#X, y = shuffle(X, y)

num_images, height, width, channels = X.shape
batch_size = 100

# Initialize empty arrays for the processed data
X_processed = np.empty_like(X, dtype=np.float32)
y_processed = np.empty_like(y)

# Process data in chunks
for start in range(0, num_images, batch_size):
    end = min(start + batch_size, num_images)
    
    # Extract the current batch
    X_batch = X[start:end]
    y_batch = y[start:end]
    
    # Normalize
    X_batch = X_batch / 255.0
    
    # Shuffle the batch
    X_batch, y_batch = shuffle(X_batch, y_batch, random_state=0)
    
    # Store the processed batch back into the main arrays
    X_processed[start:end] = X_batch
    y_processed[start:end] = y_batch

# Overwrite the original arrays with the processed data
X = X_processed
y = y_processed

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
def build_cnn(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
cnn_model = build_cnn(X_train.shape[1:])
cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

/home/caioprimola/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-08-25 19:08:38.890362: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 306380800 exceeds 10% of free system memory.
2024-08-25 19:08:38.955749: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 306380800 exceeds 10% of free system memory.
2024-08-25 19:08:39.064516: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 306380800 exceeds 10% of free system memory.


: 

In [11]:
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test, y_test)
cnn_predictions = np.argmax(cnn_model.predict(X_test), axis=-1)
cnn_precision = precision_score(y_test, cnn_predictions, average='weighted')

13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 877ms/step - accuracy: 0.6992 - loss: 0.6244
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 858ms/step


/home/caioprimola/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_flat, y_train)
knn_predictions = knn.predict(X_test_flat)
knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_precision = precision_score(y_test, knn_predictions, average='weighted')

: 

In [8]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_flat, y_train)
rf_predictions = rf.predict(X_test_flat)
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions, average='weighted')

In [9]:
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_flat, y_train)
lr_predictions = lr.predict(X_test_flat)
lr_accuracy = accuracy_score(y_test, lr_predictions)
lr_precision = precision_score(y_test, lr_predictions, average='weighted')

In [10]:
nb = GaussianNB()
nb.fit(X_train_flat, y_train)
nb_predictions = nb.predict(X_test_flat)
nb_accuracy = accuracy_score(y_test, nb_predictions)
nb_precision = precision_score(y_test, nb_predictions, average='weighted')

In [11]:
print(f"CNN - Accuracy: {cnn_accuracy:.4f}, Precision: {cnn_precision:.4f}")
print(f"KNN - Accuracy: {knn_accuracy:.4f}, Precision: {knn_precision:.4f}")
print(f"Random Forest - Accuracy: {rf_accuracy:.4f}, Precision: {rf_precision:.4f}")
print(f"Logistic Regression - Accuracy: {lr_accuracy:.4f}, Precision: {lr_precision:.4f}")
print(f"Naive Bayes - Accuracy: {nb_accuracy:.4f}, Precision: {nb_precision:.4f}")

Random Forest - Accuracy: 0.8195, Precision: 0.8154
Logistic Regression - Accuracy: 0.8120, Precision: 0.8059
Naive Bayes - Accuracy: 0.7544, Precision: 0.7884
